In [1]:
from pytube import Channel, YouTube, Playlist
from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api.formatters import TextFormatter
import pandas as pd

/Users/karthickmani/Documents/Python/youtube_recipes_llm/env/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# Popular Ranveer Brar Recipes Playlist. Using playlist instead of channels since pytube does not support new channel format
rb_playlist = Playlist('https://www.youtube.com/watch?v=zAICu_dIlUs&list=UULPEHCDn_BBnk3uTK1M64ptyw')
formatter = TextFormatter()

In [ ]:
df = pd.DataFrame()

for video in rb_playlist:
    d = {}
    v_id = video.split('watch?v=')[1]
    d['video_id'] = v_id
    d['title'] = YouTube(video).title
    d['publish_date'] = YouTube(video).publish_date
    d['length'] = YouTube(video).length
    d['thumbnail'] = YouTube(video).thumbnail_url
    d['vid_info'] = YouTube(video).vid_info
    
    try:
        
        #transcript_list = YouTubeTranscriptApi.list_transcripts(v_id)
        transcript = YouTubeTranscriptApi.get_transcript(v_id, languages=['en', 'en-GB', 'hi', 'en-IN'])
        d['transcript'] = formatter.format_transcript(transcript)
    except:
        print('No transcripts available for ', v_id)
        d['transcript'] = 'NA'
        continue

    df = pd.concat([df, pd.DataFrame([d])], ignore_index=True)

In [ ]:
df.head(5)

In [ ]:
df.to_csv('../data/recipes.csv', index=False)

In [ ]:
def get_videos_df_from_channel(channel_url):
    channel = Channel(channel_url)
    df = pd.DataFrame()
    print(f'There are {len(channel.videos)} videos in {channel.channel_name}')

    
    for idx, video in (enumerate(channel.videos)):
        if idx%10 == 0:
            print(f'processing {idx}/{len(channel.videos)} video')
        
        d = {}
        d['video_id'] = video.video_id
        d['title'] = video.title
        d['publish_date'] = video.publish_date
        d['length'] = video.length
        d['thumbnail'] = video.thumbnail_url
        d['vid_info'] = video.vid_info
        
        try:
            
            #transcript_list = YouTubeTranscriptApi.list_transcripts(v_id)
            transcript = YouTubeTranscriptApi.get_transcript(video.video_id, languages=['en-IN'])
            d['transcript'] = formatter.format_transcript(transcript)
        except:
            print('No transcripts available for ', video.video_id)
            d['transcript'] = 'NA'
            continue

        df = pd.concat([df, pd.DataFrame([d])], ignore_index=True)
    return df

In [ ]:
# Using pytubefix package to use new channel urls. Downloading transcripts for Venkatesh Bhat's recipes

from pytubefix import YouTube, Channel

vb_df = get_videos_df_from_channel('https://www.youtube.com/@venkateshbhatsidhayamthott3527')

In [ ]:
vb_df.to_csv('../data/recipes_vb.csv', index=False)